In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import glob
import time
import pickle

EPSILON = 1e-10

In [14]:
def get_iso_data(iso_data_path):
	csv_files = glob.glob(f'{iso_data_path}/*.csv')
	filtered_csv_files = [file for file in csv_files if not file.endswith('_info.csv')]
	#print(filtered_csv_files[0:5])
	df = pd.concat([pd.read_csv(file) for file in filtered_csv_files], ignore_index = True)
	# first innings data
	df = df[df['innings']==1]
	df.to_csv('../data/2001to24.csv', index = False)
	#print(df[0:5])
	df = df[['match_id', 'start_date', 'innings', 'ball', 'runs_off_bat', 'extras', 'wicket_type']]
	df['wickets_indicator'] = df['wicket_type'].notna().astype(int)
	df['balls_remaining'] = 300 - ((df['ball'].astype(int) * 6) + ((df['ball'] * 10) % 10).astype(int))
		
	# Calculate wickets remaining for each match and innings
	df['wickets_fallen'] = df.groupby(['match_id', 'innings'])['wickets_indicator'].cumsum()
	df['wickets_remaining'] = 10 - df['wickets_fallen']

	#print(df[270:300])
	return df

'''
iso_df = get_iso_data('../data/recently_added_2_male_csv2')
sub_df = iso_df[['match_id', 'start_date', 'innings', 'balls_remaining', 'wickets_remaining', 'wickets_indicator']]
sub_df.head()
'''
df = get_iso_data('../data/recently_added_2_male_csv2')
print("Columns in DataFrame:", df.columns)


Columns in DataFrame: Index(['match_id', 'start_date', 'innings', 'ball', 'runs_off_bat', 'extras',
       'wicket_type', 'wickets_indicator', 'balls_remaining', 'wickets_fallen',
       'wickets_remaining'],
      dtype='object')


# The Components of the distribution function

## Probability of extras

In [15]:
def get_prob_of_wide_or_no_ball(iso_df):
	extras_runs = iso_df['extras'].sum()
	total_no_of_balls = iso_df['ball'].count()
	prob_of_wide_or_no_ball = extras_runs/(total_no_of_balls+extras_runs)
	print(f'extras_runss: {extras_runs}, total_no_of_balls:{total_no_of_balls}')
	return prob_of_wide_or_no_ball

## Wicket Process

In [16]:
class WicketProbit:
    def __init__(self, df):
        self.df = df
        #Initial guesses
        self.a0 = 1.67
        self.a1 = 0.00758
        self.a2 = -0.0459
        self.a3 = -0.0000160
        self.res = None
    
    def LLF(self, params):
        a0, a1, a2, a3 = params
        x = -a0 - a1*self.df['balls_remaining'] - a2*self.df['wickets_remaining'] - a3*(self.df['balls_remaining']**2)
        llf_col = self.df['wickets_indicator']*np.log(norm.cdf(x) + EPSILON) + (1-self.df['wickets_indicator'])*np.log(1-norm.cdf(x) + EPSILON)
        sum = np.sum(llf_col)
        return -sum
    
    def fit(self):
        res = minimize(self.LLF, [self.a0, self.a1, self.a2, self.a3])
        self.a0, self.a1, self.a2, self.a3 = res.x
        self.res = res
        return None
    
    def predict(self, balls_remaining, wickets_remaining):
        x = -self.a0 - self.a1*balls_remaining - self.a2*wickets_remaining - self.a3*(balls_remaining**2)
        return norm.cdf(x)

## Runs Process

In [17]:
def compute_llf(val, params, mu0):
    a0, a1, a2, a3, mu1, mu2, mu3, mu4, mu5 = params
    if val==0:
        llf = np.log(norm.cdf((mu0-val))+ EPSILON)
    elif val==1:
        llf = np.log(norm.cdf((mu1-val)) - norm.cdf((mu0-val))+ EPSILON)
    elif val==2:
        llf = np.log(norm.cdf((mu2-val)) - norm.cdf((mu1-val))+ EPSILON)
    elif val==3:
        llf = np.log(norm.cdf((mu3-val)) - norm.cdf((mu2-val))+ EPSILON)
    elif val==4:
        llf = np.log(norm.cdf((mu4-val)) - norm.cdf((mu3-val))+ EPSILON)
    elif val==5:
        llf = np.log(norm.cdf((mu5-val)) - norm.cdf((mu4-val))+ EPSILON)
    else:
        llf = np.log(1-norm.cdf((mu5-val))+ EPSILON)
    # it was raising warning because it was trying to compute -ve value for log
    # this should resolve the issue
    if llf<=0:
            llf = EPSILON
    return llf

In [18]:
class RunsOProbit:
    def __init__(self, df):
        self.df = df
        #Initial guesses
        self.a0 = -0.174
        self.a1 = -0.00844
        self.a2 = 0.130
        self.a3 = 0.0000106
        #Ordered thresholds
        self.mu0 = 0 #Symbolically defined, we set to 0 wlg
        self.mu1 = 0.940
        self.mu2 = 1.263
        self.mu3 = 1.325
        self.mu4 = 2.321
        self.mu5 = 2.328
        self.res = None
        
    def LLF(self, params):
        a0, a1, a2, a3, mu1, mu2, mu3, mu4, mu5 = params
        mu0 = self.mu0
        # get only the rows where wickets have not fallen
        col = self.df[self.df['wickets_indicator']==0]
        
        # compute the value of x
        x = a0 + a1*col['balls_remaining'] + a2*col['wickets_remaining'] + a3*(col['balls_remaining']**2)

        # compute the llf for each run value
        llf_col = x.apply(lambda val: compute_llf(val, params, mu0))
        sum = llf_col.sum()
        
        return -sum
    
    def fit(self):
        res = minimize(self.LLF, [self.a0, self.a1, self.a2, self.a3, self.mu1, self.mu2, self.mu3, self.mu4, self.mu5])
        self.a0, self.a1, self.a2, self.a3, self.mu1, self.mu2, self.mu3, self.mu4, self.mu5 = res.x
        self.res = res
        return None
    
    def predict(self, runs, balls_remaining, wickets_remaining):
        x = self.a0 + self.a1*balls_remaining + self.a2*wickets_remaining + self.a3*(balls_remaining**2)
        if runs == 0:
            return norm.cdf(self.mu0-x)
        elif runs == 1:
            return norm.cdf(self.mu1-x) - norm.cdf(self.mu0-x)
        elif runs == 2:
            return norm.cdf(self.mu2-x) - norm.cdf(self.mu1-x)
        elif runs == 3:
            return norm.cdf(self.mu3-x) - norm.cdf(self.mu2-x)
        elif runs == 4:
            return norm.cdf(self.mu4-x) - norm.cdf(self.mu3-x)
        elif runs == 5:
            return norm.cdf(self.mu5-x) - norm.cdf(self.mu4-x)
        else:
            return 1-norm.cdf(self.mu5-x)

# Constructing the distribution function

In [19]:
def construct_F(max_runs, max_balls, max_wickets, wicket_model, runs_model, p_extra):
    F = np.zeros((max_runs+1, max_balls+1, max_wickets+1))
    
    #Boundary conditions
    F[:,0,:] = 1 #No balls left, so win is impossible
    F[:,:,0] = 1 #No wickets left, so win is impossible
    
    #Filling in the rest of the table using the recursion
    for b in range(1, max_balls+1):
        for w in range(1, max_wickets+1):
            wicket_prob = wicket_model.predict(b, w)
            runs_0_prob = runs_model.predict(0,b,w)
            runs_1_prob = runs_model.predict(1,b,w)
            runs_2_prob = runs_model.predict(2,b,w)
            runs_3_prob = runs_model.predict(3,b,w)
            runs_4_prob = runs_model.predict(4,b,w)
            runs_5_prob = runs_model.predict(5,b,w)
            runs_6_prob = runs_model.predict(6,b,w)
            for r in range(max_runs+1):
                term1 = p_extra*F[r-1,b,w] if r>0 else 0 #Extra ball
                term2 = (1-p_extra)*wicket_prob*F[r,b-1,w-1] #Wicket
                #term3 = (1-p_extra)*(1-wicket_model.predict(b, w))*sum([runs_model.predict(i, b, w)*F[r-i,b-1,w] for i in range(7)]) #Runs
                
                #To accomodate for the cases when the runs scored can be greater than the remaining runs
                term3 = 0
                if r >= 0:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_0_prob*F[r-0,b-1,w] #Runs
                if r >= 1:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_1_prob*F[r-1,b-1,w] #Runs
                if r >= 2:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_2_prob*F[r-2,b-1,w] #Runs
                if r >= 3:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_3_prob*F[r-3,b-1,w] #Runs
                if r >= 4:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_4_prob*F[r-4,b-1,w] #Runs
                if r >= 5:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_5_prob*F[r-5,b-1,w] #Runs
                if r >= 6:
                    term3 += (1-p_extra)*(1-wicket_prob)*runs_6_prob*F[r-6,b-1,w] #Runs
                
                F[r,b,w] = term1 + term2 + term3
    
    return F

# *sighs* Training 

In [20]:
def training(data_path):
    #import data
    time1 = time.time()
    dataframe = get_iso_data(data_path)
    time2 = time.time()

    print(f'preprocessing time: {time2 - time1}')
    #Setup the three processes
    px = get_prob_of_wide_or_no_ball(dataframe) #Probability of wide or no ball
    
    wicket_model = WicketProbit(dataframe) #Wicket model
    print('fitting...')

    time3 = time.time()
    wicket_model.fit()
    time4 = time.time()
    print("Wicket process fit!")
    print(f'wicket process fitting time: {time4 - time3}')
    

    runs_model = RunsOProbit(dataframe) #Runs model
    print('fitting...')

    time5 = time.time()
    runs_model.fit()
    time6 = time.time()
    print("Runs process fit!")
    print(f'Runs process fitting time: {time6 - time5}')
    
    
    #Construct the F table
    max_runs = 500
    max_balls = 300
    max_wickets = 10

    time7 = time.time()
    print(f'constructing F ...')
    F = construct_F(max_runs, max_balls, max_wickets, wicket_model, runs_model, px)
    time8 = time.time()
    print(f'F construction time: {time8 - time7}')

    print(f'Total execution time: {time8 - time1}')
    
    return F, wicket_model, runs_model, px

In [21]:
data_path = "../data/odis_male_csv2"
F, wicket_model, runs_model, px = training(data_path)

preprocessing time: 17.284932613372803
extras_runss: 34934, total_no_of_balls:726239
fitting...
Wicket process fit!
wicket process fitting time: 23.52197527885437
fitting...
Runs process fit!
Runs process fitting time: 543.7909910678864
constructing F ...
F construction time: 11.997541904449463
Total execution time: 596.5974431037903


In [23]:
#Inspect the models and their LLF's
print("Wickets process:")
final_llf = -wicket_model.LLF(wicket_model.res.x)
print(f'Final Log-Likelihood: {final_llf}')
print('Final Parameters:')
print(f'a0: {wicket_model.a0}, a1: {wicket_model.a1}, a2: {wicket_model.a2}, a3: {wicket_model.a3}')
print('---------------------------------------------------------------------------------------------')
print("Runs process:")
final_llf = -runs_model.LLF(runs_model.res.x)
print(f'Final Log-Likelihood: {final_llf}')
print('Final Parameters:')
print(f'a0: {runs_model.a0}, a1: {runs_model.a1}, a2: {runs_model.a2}, a3: {runs_model.a3}')
print(f'mu0: {runs_model.mu0}, mu1: {runs_model.mu1}, mu2: {runs_model.mu2}, mu3: {runs_model.mu3}, mu4: {runs_model.mu4}, mu5: {runs_model.mu5}')
print('---------------------------------------------------------------------------------------------')

Wickets process:
Final Log-Likelihood: -85000.44274791362
Final Parameters:
a0: 0.8485612612059278, a1: 0.003491470865920923, a2: 0.1775310582050424, a3: -1.919091993089362e-05
---------------------------------------------------------------------------------------------
Runs process:
Final Log-Likelihood: 7.062399999999997e-05
Final Parameters:
a0: -0.174, a1: -0.00844, a2: 0.13, a3: 1.06e-05
mu0: 0, mu1: 0.94, mu2: 1.263, mu3: 1.325, mu4: 2.321, mu5: 2.328
---------------------------------------------------------------------------------------------


In [27]:
table4 = np.zeros((8, 4))
cases = [[300,10],[120,10],[120,3],[6,3]]
for i in range(4):
    balls_left = cases[i][0]
    wickets_left = cases[i][1]
    x = -wicket_model.a0 - wicket_model.a1*balls_left - wicket_model.a2*wickets_left - wicket_model.a3*(balls_left**2)
    y = runs_model.a0 + runs_model.a1*balls_left + runs_model.a2*wickets_left + runs_model.a3*(balls_left**2)
    table4[0,i] = (1-px)*(norm.cdf(x))
    table4[1,i] = (1-px)*(1-norm.cdf(x))*(norm.cdf(runs_model.mu0-y))
    table4[2,i] = (1-px)*(1-norm.cdf(x))*(norm.cdf(runs_model.mu1-y) - norm.cdf(runs_model.mu0-y))
    table4[3,i] = (1-px)*(1-norm.cdf(x))*(norm.cdf(runs_model.mu2-y) - norm.cdf(runs_model.mu1-y))
    table4[4,i] = (1-px)*(1-norm.cdf(x))*(norm.cdf(runs_model.mu3-y) - norm.cdf(runs_model.mu2-y))
    table4[5,i] = (1-px)*(1-norm.cdf(x))*(norm.cdf(runs_model.mu4-y) - norm.cdf(runs_model.mu3-y))
    table4[6,i] = (1-px)*(1-norm.cdf(x))*(norm.cdf(runs_model.mu5-y) - norm.cdf(runs_model.mu4-y))
    table4[7,i] = (1-px)*(1-norm.cdf(x))*(1-norm.cdf(runs_model.mu5-y))
    
print(pd.DataFrame(table4, columns = ['Case 1', 'Case 2', 'Case 3', 'Case 4'], index = ['0', '1', '2', '3', '4', '5', '6', '7']))

     Case 1    Case 2    Case 3    Case 4
0  0.024749  0.002703  0.060854  0.076849
1  0.626725  0.375976  0.661241  0.380887
2  0.226459  0.337476  0.181470  0.303909
3  0.036048  0.086351  0.025305  0.072922
4  0.005008  0.013871  0.003379  0.011482
5  0.032534  0.118765  0.020504  0.094397
6  0.000055  0.000319  0.000030  0.000238
7  0.002526  0.018644  0.001321  0.013421


# Saving the final parameters

In [ ]:
np.save('../model/F.npy', F)
final_wicket_params = {'a0': wicket_model.a0, 'a1': wicket_model.a1, 'a2': wicket_model.a2, 'a3': wicket_model.a3}
final_runs_params = {'a0': runs_model.a0, 'a1': runs_model.a1, 'a2': runs_model.a2, 'a3': runs_model.a3, 'mu1': runs_model.mu1, 'mu2': runs_model.mu2, 'mu3': runs_model.mu3, 'mu4': runs_model.mu4, 'mu5': runs_model.mu5}
np.save('../model/px.npy', px)
with open('../model/wicket_params.pkl', 'wb') as file:
    pickle.dump(final_wicket_params, file)
with open('../model/runs_params.pkl', 'wb') as file:
    pickle.dump(final_runs_params, file)

# Higher level functions

In [ ]:
def game_state_info(F, runs_model, wicket_model, px, runs_remaining, balls_left, wickets_left):
    #Computes the probabilities of various next ball outcomes given a certain game state in the second Innings
    win_prob = 1 - F[runs_remaining, balls_left, wickets_left]
    x = -wicket_model.a0 - wicket_model.a1*balls_left - wicket_model.a2*wickets_left - wicket_model.a3*(balls_left**2)
    y = runs_model.a0 + runs_model.a1*balls_left + runs_model.a2*wickets_left + runs_model.a3*(balls_left**2)
    prob_extra = px
    prob_wicket = (1 - px)*(norm.cdf(x))
    prob_0run = (1 - px)*(1 - norm.cdf(x))*(norm.cdf(runs_model.mu0-y))
    prob_1run = (1 - px)*(1 - norm.cdf(x))*(norm.cdf(runs_model.mu1-y) - norm.cdf(runs_model.mu0-y))
    prob_2run = (1 - px)*(1 - norm.cdf(x))*(norm.cdf(runs_model.mu2-y) - norm.cdf(runs_model.mu1-y))
    prob_3run = (1 - px)*(1 - norm.cdf(x))*(norm.cdf(runs_model.mu3-y) - norm.cdf(runs_model.mu2-y))
    prob_4run = (1 - px)*(1 - norm.cdf(x))*(norm.cdf(runs_model.mu4-y) - norm.cdf(runs_model.mu3-y))
    prob_5run = (1 - px)*(1 - norm.cdf(x))*(norm.cdf(runs_model.mu5-y) - norm.cdf(runs_model.mu4-y))
    prob_6run = (1 - px)*(1 - norm.cdf(x))*(1 - norm.cdf(runs_model.mu5 - y))
    
    return win_prob, prob_extra, prob_wicket, prob_0run, prob_1run, prob_2run, prob_3run, prob_4run, prob_5run, prob_6run

def find_modified_target(F, runs_remaining, balls_left_init, wickets_left, balls_left_fin):
    #Computes the adjusted runs to get when the second innings is shortened
    reference = F[runs_remaining, balls_left_init, wickets_left]
    
    closest_target = None
    min_diff = float('inf')
    
    max_r = F.shape[0]
    for r in range(max_r):
        diff = abs(F[r,balls_left_fin,wickets_left] - reference)
        
        if diff < min_diff:
            min_diff = diff
            closest_target = r
            
    return closest_target